In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import torch

In [7]:
from proteome.models.folding.esm import config

In [8]:
names = ["esmfold_3B_v0", "esmfold_3B_v1"]

In [9]:
cfgs = {}
for name in names:
    url = f"https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t36_3B_UR50D.pt"
    model_data = torch.hub.load_state_dict_from_url(
        url, progress=False, map_location="cpu"
    )
    cfgs[name] = model_data["cfg"]["model"]

In [15]:
#cfgs["esmfold_3B_v0"]

In [37]:
def compare_dicts(dict1, dict2):
    missing = []
    different = []
    for k,v in dict1.items():
        if k not in dict2:
            missing.append(k)
        elif isinstance(v, dict):
            compare_dicts(v, dict2[k])
        else:
            if dict2[k] != v:
                different.append(k)

    print("Missing", missing)
    print("Different", different)

In [38]:
compare_dicts(cfgs["esmfold_3B_v0"], cfgs["esmfold_3B_v1"])

Missing []
Different []


In [31]:
for k,v in cfgs["esmfold_3B_v0"]:
    if not isinstance()

dict_keys(['esmfold_3B_v0', 'esmfold_3B_v1'])

In [16]:
getattr(config.StructureModuleConfig(c_s=512), "c_s")

512

In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import os
import py3Dmol
import torch
from proteome import protein
from proteome.models.folding.esm.modeling import ESMForFolding
from proteome.models.folding.openfold.utils.feats import atom14_to_atom37

In [3]:
sequence = 'MAAHKGAEHHHKAAEHHEQAAKHHHAAAEHHEKGEHEQAAHHADTAYAHHKHAEEHAAQAAKHDAEHHAPKPH'

In [13]:
folder = ESMForFolding(chunk_size=512)
output = folder.fold(sequence)

In [4]:
model = pretrained.esmfold_v0()
model = model.half()
model = model.eval()
model.set_chunk_size(512)

/home/conradry71/micromamba/envs/proteome/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model = model.cuda()

In [6]:
output = model.infer([sequence])

In [8]:
final_atom_positions = atom14_to_atom37(output["positions"][-1], output)
output = {k: v.to("cpu").numpy() for k, v in output.items()}
final_atom_positions = final_atom_positions.cpu().numpy()
final_atom_mask = output["atom37_atom_exists"]

In [9]:
for k,v in output.items():
    output[k] = v.squeeze()

In [10]:
predicted_protein = protein.Protein(
    aatype=output["aatype"],
    atom_positions=final_atom_positions.squeeze(),
    atom_mask=final_atom_mask.squeeze(),
    residue_index=output["residue_index"] + 1,
    b_factors=output["plddt"],
    chain_index=output["chain_index"] if "chain_index" in output else None,
)

In [11]:
result_pdb = protein.to_pdb(predicted_protein)

In [12]:
PLDDT_BANDS = [
  (0, 50, '#FF7D45'),
  (50, 70, '#FFDB13'),
  (70, 90, '#65CBF3'),
  (90, 100, '#0053D6')
]
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(result_pdb)
color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}

style['stick'] = {}

view.setStyle({'model': -1}, style)
view.zoomTo()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol